In [49]:
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
import string
import collections
import random
import time
import nltk
from nltk import word_tokenize
from textaugment import Wordnet

In [50]:
local_api_url = 'http://localhost:8080/resource/skill'

In [ ]:
# Not needed for now, just saved here in case it's needed for later -- also Silenced

# search for all jobs that have 'manager' in their title  --- IGNORE this cell for now
# WEB API URL (in case you don't want to use the local API): url_title = "https://ec.europa.eu/esco/api/search"
# also: url = "https://ec.europa.eu/esco/api/resource/skill"

params = {
    'language': 'en',
    'type': 'occupation',
    'text': 'pig'
         }

api_title = requests.get(url=url_title, params=params).json()

In [ ]:
# create an augmented dataframe by appending rows for each alternative label of each job title
# and uses the Text Augmentation model to create synonym descriptions for each alternative label
# DON'T RUN THIS, IT'S BEEN RUN ALREADY -- it can only to be run once because it takes a while :)
# (also, this cell has been set to Silence)

# get word thesaurus

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# initialize augment model

t = Wordnet(v=False ,n=True, p=0.75)

# run the API sweeping loop

df_occupations_aug = df_occupations

for index, row in df_occupations.iterrows():

    if pd.notna(df_occupations.iloc[index,4]):
        
        list_of_altLabels_per_row = row[4].split('\n')        
        
        for altLabel in list_of_altLabels_per_row:
            
            df_of_separate_row = pd.DataFrame([row])
            df_occupations_aug = pd.concat([df_occupations_aug, df_of_separate_row])
            print(f"Job Title {index}: Appended {row[3]}'s alternative label: {altLabel}")
            df_occupations_aug.iloc[-1,3] = altLabel
            df_occupations_aug.iloc[-1,12] = t.augment(row[12])

# write augmented occupations dataframe to CSV file
df_occupations_aug.to_csv('../data/ESCO/occupations_augmented.csv')

In [51]:
df_occupations_aug = pd.read_csv('../data/ESCO/occupations_augmented.csv')

In [52]:
df_occupations_aug['skills'] = 0
df_occupations_aug['opt_skills'] = 0

In [53]:
df_occupations_aug.head()

,Unnamed: 0,conceptType,conceptUri,iscoGroup,preferredLabel,altLabels,hiddenLabels,status,modifiedDate,regulatedProfessionNote,scopeNote,definition,inScheme,description,code,skills,opt_skills
0,0,Occupation,http://data.europa.eu/esco/occupation/00030d09...,2654,technical director,technical and operations director\nhead of tec...,NaN,released,2016-07-05T13:58:41Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Technical directors realise the artistic visio...,2654.1.7,0,0
1,1,Occupation,http://data.europa.eu/esco/occupation/000e93a3...,8121,metal drawing machine operator,metal drawing machine operator\nmetal drawing ...,NaN,released,2016-07-05T17:09:43Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Metal drawing machine operators set up and ope...,8121.4,0,0
2,2,Occupation,http://data.europa.eu/esco/occupation/0019b951...,7543,precision device inspector,inspector of precision instruments\nprecision ...,NaN,released,2016-07-06T09:21:20Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Precision device inspectors make sure precisio...,7543.10.3,0,0
3,3,Occupation,http://data.europa.eu/esco/occupation/0022f466...,3155,air traffic safety technician,air traffic safety electronics hardware specia...,NaN,released,2017-01-17T11:40:37Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Air traffic safety technicians provide technic...,3155.1,0,0
4,4,Occupation,http://data.europa.eu/esco/occupation/002da35b...,2431,hospitality revenue manager,hospitality revenues manager\nyield manager\nh...,NaN,released,2017-01-17T13:33:42Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Hospitality revenue managers maximise revenue ...,2431.9,0,0


In [55]:
### create a separate skill column on the augmented dataframe for the skills got from the local API

for index, row in tqdm(df_occupations_aug.iterrows()):
    
    params = {'uri': df_occupations_aug.iloc[index,2], 'language': 'en'}
    api_skills = requests.get(url=local_api_url, params=params).json()
    api_skills_dict = api_skills.get('_links').get('hasEssentialSkill')
    if api_skills.get('_links').get('hasOptionalSkill') != None:
        api_opt_skills_dict = api_skills.get('_links').get('hasOptionalSkill')
    
    try:
        skills_list = [skill.get('title') for skill in api_skills_dict]
        opt_skills_list = [skill.get('title') for skill in api_opt_skills_dict]
    except:
        pass
        
    df_occupations_aug.iloc[index,15] = ', '.join(skills_list) 
    df_occupations_aug.iloc[index,16] = ', '.join(opt_skills_list) 

35824it [56:44, 10.52it/s]


In [56]:
df_occupations_aug.to_csv('../data/ESCO/occupations_augmented_with_OPTIONAL_SKILLS.csv')


In [7]:
params = {'uri': df_occupations_aug.iloc[index,2], 'language': 'en'}
requests.get(url=local_api_url, params=params).url

'http://localhost:8080/resource/skill?uri=http%3A%2F%2Fdata.europa.eu%2Fesco%2Foccupation%2F0eebf6d2-af43-4a18-b4b7-668fb6433b42&language=en'